In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import time

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

# Initialize and train autoencoder
autoencoder = Autoencoder().to(device)
ae_criterion = nn.MSELoss()
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
num_epochs_ae = 10

print("Training Autoencoder...")
start_time_ae = time.time()
for epoch in range(num_epochs_ae):
    autoencoder.train()
    running_loss = 0
    for i, (images, _) in enumerate(train_loader):
        images = images.to(device)
        ae_optimizer.zero_grad()
        _, outputs = autoencoder(images)
        loss = ae_criterion(outputs, images)
        loss.backward()
        ae_optimizer.step()
        running_loss += loss.item()

        if (i + 1) % 150 == 0 or (i + 1) == len(train_loader):
            print(f"Epoch [{epoch + 1}/{num_epochs_ae}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}")
            running_loss = 0

print(f"Autoencoder Training Completed in {time.time() - start_time_ae:.2f} seconds")

# Save trained autoencoder
torch.save(autoencoder.state_dict(), 'autoencoder.pth')
print("Autoencoder saved as 'autoencoder.pth'")


Training Autoencoder...
Epoch [1/10], Step [150/391], Loss: 0.1759
Epoch [1/10], Step [300/391], Loss: 0.0791
Epoch [1/10], Step [391/391], Loss: 0.0400
Epoch [2/10], Step [150/391], Loss: 0.0566
Epoch [2/10], Step [300/391], Loss: 0.0483
Epoch [2/10], Step [391/391], Loss: 0.0267
Epoch [3/10], Step [150/391], Loss: 0.0410
Epoch [3/10], Step [300/391], Loss: 0.0379
Epoch [3/10], Step [391/391], Loss: 0.0221
Epoch [4/10], Step [150/391], Loss: 0.0345
Epoch [4/10], Step [300/391], Loss: 0.0324
Epoch [4/10], Step [391/391], Loss: 0.0194
Epoch [5/10], Step [150/391], Loss: 0.0313
Epoch [5/10], Step [300/391], Loss: 0.0298
Epoch [5/10], Step [391/391], Loss: 0.0175
Epoch [6/10], Step [150/391], Loss: 0.0285
Epoch [6/10], Step [300/391], Loss: 0.0279
Epoch [6/10], Step [391/391], Loss: 0.0166
Epoch [7/10], Step [150/391], Loss: 0.0268
Epoch [7/10], Step [300/391], Loss: 0.0258
Epoch [7/10], Step [391/391], Loss: 0.0155
Epoch [8/10], Step [150/391], Loss: 0.0255
Epoch [8/10], Step [300/391], 

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import time

# Define the Classifier
class Classifier(nn.Module):
    def __init__(self, encoder):
        super(Classifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 2 * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        features = self.encoder(x)
        output = self.classifier(features)
        return output

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Load trained autoencoder
autoencoder = Autoencoder().to(device)
autoencoder.load_state_dict(torch.load('autoencoder.pth'))
autoencoder.eval()

# Initialize classifier
classifier = Classifier(autoencoder.encoder).to(device)
clf_criterion = nn.CrossEntropyLoss()
clf_optimizer = optim.Adam(classifier.parameters(), lr=0.001)
num_epochs_clf = 20

print("Training Classifier...")
start_time_clf = time.time()
for epoch in range(num_epochs_clf):
    classifier.train()
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        clf_optimizer.zero_grad()
        outputs = classifier(images)
        loss = clf_criterion(outputs, labels)
        loss.backward()
        clf_optimizer.step()
        total_loss += loss.item()

        if (i + 1) % 150 == 0 or (i + 1) == len(train_loader):
            print(f"Epoch [{epoch + 1}/{num_epochs_clf}], Step [{i + 1}/{len(train_loader)}], Loss: {total_loss / 100:.4f}")
            total_loss = 0

print(f"Classifier Training Completed in {time.time() - start_time_clf:.2f} seconds")

# Evaluate classifier
print("Evaluating Classifier...")
classifier.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = classifier(images)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples * 100
print(f"Classifier Accuracy: {accuracy:.2f}%")


Training Classifier...
Epoch [1/20], Step [150/391], Loss: 2.4579
Epoch [1/20], Step [300/391], Loss: 1.8643
Epoch [1/20], Step [391/391], Loss: 1.0142
Epoch [2/20], Step [150/391], Loss: 1.5134
Epoch [2/20], Step [300/391], Loss: 1.4227
Epoch [2/20], Step [391/391], Loss: 0.8017
Epoch [3/20], Step [150/391], Loss: 1.2179
Epoch [3/20], Step [300/391], Loss: 1.1421
Epoch [3/20], Step [391/391], Loss: 0.6912
Epoch [4/20], Step [150/391], Loss: 1.0099
Epoch [4/20], Step [300/391], Loss: 1.0026
Epoch [4/20], Step [391/391], Loss: 0.6131
Epoch [5/20], Step [150/391], Loss: 0.8335
Epoch [5/20], Step [300/391], Loss: 0.8327
Epoch [5/20], Step [391/391], Loss: 0.5185
Epoch [6/20], Step [150/391], Loss: 0.6987
Epoch [6/20], Step [300/391], Loss: 0.7153
Epoch [6/20], Step [391/391], Loss: 0.4328
Epoch [7/20], Step [150/391], Loss: 0.5751
Epoch [7/20], Step [300/391], Loss: 0.6042
Epoch [7/20], Step [391/391], Loss: 0.3538
Epoch [8/20], Step [150/391], Loss: 0.4534
Epoch [8/20], Step [300/391], L